In [ ]:
%pip install taichi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import taichi.math as tm
ti.init(arch=ti.cpu)

[Taichi] version 1.7.2, llvm 15.0.4, commit 0131dce9, linux, python 3.10.12
[Taichi] Starting on arch=x64


Typecasting inside func

Here are a few examples for a very basic operation:

We are given an integer x, and we want to add 0.5 to that integer (similar to centering pixels in A1)

The issue is, x is an integer, so we need to convert the operation to floats

In [ ]:
@ti.func
def simple_add_1(x: int) -> float:
  x = x + 0.5
  return float(x)

@ti.kernel
def k1() -> float:
  x = simple_add_1(1)
  return x

print(k1())

1.0


Issue #1: Storing x + 0.5 directly in x: x is an integer, so this operation will get cast into an int

In [ ]:
@ti.func
def simple_add_2(x: int) -> float:
  x = float(x + 0.5)
  return float(x)

@ti.kernel
def k2() -> float:
  x = simple_add_2(1)
  return x

print(k2())

1.0


Issue #2: Type casting the operation will NOT make x a float

In [ ]:
@ti.func
def simple_add_3(x: int) -> float:
  x = float(x) + 0.5
  return float(x)

@ti.kernel
def k3() -> float:
  x = simple_add_3(1)
  return x

print(k3())

1.0


Issue #3: Type casting x before the operation will make x a float

In [ ]:
@ti.func
def simple_add_4(x: int) -> float:
  x = ti.cast(x + 0.5, ti.f32)
  return float(x)

@ti.kernel
def k4() -> float:
  x = simple_add_4(1)
  return x

print(k4())

1.0


The same issue appears when you use the ti.cast function on the operation

In [ ]:
@ti.func
def simple_add_5(x: int) -> float:
  x = ti.cast(x, ti.f32) + 0.5
  return float(x)

@ti.kernel
def k5() -> float:
  x = simple_add_5(1)
  return x

print(k5())

1.0


As well as x itself. X WILL NOT CHANGE TYPES

In [ ]:
@ti.func
def simple_add_6(x: int) -> float:
  return x + 0.5

@ti.kernel
def k6() -> float:
  x = simple_add_6(1)
  return x

print(k6())

1.5


Solution #1: Return directly x + 0.5. Returns create new variables and do not change the type of x

In [ ]:
@ti.func
def simple_add_7(x: int) -> float:
  return float(x) + 0.5

@ti.kernel
def k7() -> float:
  x = simple_add_7(1)
  return x

print(k7())

1.5


Type casting or not, this will work

In [ ]:
@ti.func
def simple_add_8(x: int) -> float:
  y = x + 0.5
  return y

@ti.kernel
def k8() -> float:
  x = simple_add_8(1)
  return x

print(k8())

1.5


Storing the intermediary result in a variable will cast that variable into a float, but not x

More reading about Taichi's type system: https://docs.taichi-lang.org/docs/type

## Progressive Renderer

The deliverable you will implement for A2 is progressive rendering, with jittered pixels

In [ ]:
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Simple mean
total = 0
count = 0

for n in numbers:
  total += n
  count += 1

simple_mean = total / count
print(simple_mean)

5.0


In the case of our progressive renderer, we don't know beforehand how many total samples we will get, and we want to see the render as we go

In [ ]:
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Progressive Average
prog_average = 0
count = 0

for n in numbers:
  count += 1
  prog_average += (n - prog_average)/count
  # prog_average = prog_average + (n - prog_average)/count
  # prog_average = prog_average + (n/count - prog_average/count)
  # prog_average = prog_average(count-1)/count + n/count

print(prog_average)

5.0


Taichi example


In [ ]:
SPP = 10000 # Sample Per Pixel -> number of frames
width = 5
height = 5
result = ti.Vector.field(n=1, dtype=float, shape=(width, height)) # 5x5 matrix
iter_counter = ti.field(dtype=float, shape=())


@ti.kernel
def progressive_average():
  # initialize a field
  iter_counter[None] += 1
  for x, y in ti.ndrange(width, height):
    # random value between [0, 1)
    val = ti.random() # In assignment, this will be shade_ray()
    result[x, y] += (val - result[x, y])/iter_counter[None]

for _ in range(SPP):
  progressive_average() # Should be around 0.5
print(result)

[[[0.49900803]
  [0.5019654 ]
  [0.49992505]
  [0.4973632 ]
  [0.5033504 ]]

 [[0.50227475]
  [0.5025625 ]
  [0.49757576]
  [0.4931165 ]
  [0.49974763]]

 [[0.49791467]
  [0.503105  ]
  [0.50440973]
  [0.4962891 ]
  [0.5032933 ]]

 [[0.49837023]
  [0.5021516 ]
  [0.4987427 ]
  [0.4964096 ]
  [0.49995744]]

 [[0.5018112 ]
  [0.50390214]
  [0.5018027 ]
  [0.49953791]
  [0.49509612]]]


In [ ]:
result.fill(0.)
iter_counter.fill(0.)
print(result)

[[[0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [0.]]]


## Constructing an Orthonormal Basis

In [ ]:
@ti.kernel
def orthonormal_basis(axis_of_alignment:tm.vec3) -> tm.mat3:

  random_vec = tm.normalize(tm.vec3([ti.random(), ti.random(), ti.random()]))

  x_axis = tm.cross(axis_of_alignment, random_vec)
  x_axis = tm.normalize(x_axis)

  y_axis = tm.cross(x_axis, axis_of_alignment)
  y_axis = tm.normalize(y_axis)


  ortho_frames = tm.mat3([x_axis, y_axis, axis_of_alignment]).transpose()

  return ortho_frames

axis_of_alignment = tm.vec3([0.0, -1.0, 0.0])
ortho = orthonormal_basis(axis_of_alignment)

In [ ]:
w = tm.vec3([0.0, 0.0, -1.0])
w = ortho @ w
print(w)

[0. 1. 0.]


Recall orthonormal basis properties:

Given an Orthonormal Transformation Matrix M:

M M^T = I

M^T = M^(-1)

The transpose of M is also its inverse

In [ ]:
w = ortho.transpose() @ w
print(w)

[ 0.  0. -1.]
